In [428]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.io.json import json_normalize
import folium
import requests
from collections import Counter, defaultdict

Once we have called the APIs and transformed our CSV file into a Dataframe we will clean and create usefull Dataframes for the analisis: 

### Companies Dataframe:

In [429]:
conn = MongoClient("localhost:27017")
conn.list_database_names()
db = conn.get_database("ironhack")
db.list_collection_names()

['restaurants', 'books', 'companies', 'countries-small']

In [430]:
companies = db.get_collection("companies")
companies.find_one({}).keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

From this dataset we will select all the companies that where founded during and after the economic crisis of 2009. Showing only the following keys:

    - name
    - founded_year (We asume the economic crisis was fully settled in 2009, although LB crashed in sept. 2008)
    - offices (nested array, to find location of the office)

In [431]:
companies.find(
        {'founded_year': {"$gt": 2008}},   
    {"name": 1, 'founded_year': 1, "offices" : 1, "_id" : 0})

In [432]:
df_companies = pd.DataFrame(companies.find(
        {'founded_year': {"$gt": 2008}},   
    {"name": 1, 'founded_year': 1, "offices" : 1, "_id" : 0}))

Next we create a new raw for each office location without loosing its initial index:

In [438]:
df_companies = df_companies.explode('offices')

KeyError: 0

In [439]:
df_companies.offices[4]

4    {'description': '', 'address1': 'Ritterstr. 12...
4    {'description': '', 'address1': 'Fischerstr. 1...
Name: offices, dtype: object

 The cities to look for:
 
    - Madrid 
    - New York 
    - San Francisco
    - London
    - Hannover

In [363]:
selected_words = ["Madrid", "New York", "San Francisco", "London", "Hannover"] 

In [436]:
def get_count_city(offices, selected_words):

    count_dict = {}

    counts = df_companies[df_companies['offices'].isin(selected_words), ['offices'].value_counts()

    for i in range(len(counts.index.tolist())):
        count_dict[counts.index.tolist()[i]] = counts[i]

    return count_dict


SyntaxError: invalid syntax (<ipython-input-436-f998449d97db>, line 7)

In [437]:
get_count_city(offices, selected_words)

NameError: name 'offices' is not defined

### Starbucks Dataframe:
